# Datasets 2017
Para cada año tenemos 3 tablas a inspeccionar, limpiar y unir:
- `title_basics`: pública y descargada de IMDb
- `title_ratings`: pública y descargada de IMDb
- `movies`: propia, escrapeada de la web de IMDb para todas las películas del año correspondiente

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
anno = 2017

In [4]:
# Tabla descargada de IMDb

title_basics = pd.read_csv('../data/imdb/title_basics.tsv', sep='\t', na_values='\\N')
print(title_basics.shape)
title_basics.head()

C:\Users\casiopa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(8084314, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,"Comedy,Short"


In [5]:
# Tabla descargada de IMDb

title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286


In [6]:
# Películas de escrapeadas de la web IMDb

movies = pd.read_csv('../data/web_imdb/movies_df_'+ str(anno) + '.csv', sep=';', na_values=["[]", "['']", "['', '']", "['', '', '']"])
print(movies.shape)
movies.head()

(17969, 20)


,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
0,tt0100275,La Telenovela Errante,NaN,2017.0,NaN,NaN,"['Raoul Ruiz', 'Valeria Sarmiento']","['Pía Rey', 'Raoul Ruiz']","['Luis Alarcón', 'Patricia Rivadeneira', 'Fran...","['Comedy', 'Drama', 'Fantasy']",['Chile'],"['Poetastros', 'RR Producciones']",6.5,NaN,NaN,Awards\n1 win & 4 nominations,NaN,"$3,624",$512,"$3,624"
1,tt0112502,Bigfoot,NaN,2017.0,NaN,NaN,['Mc Jones'],NaN,NaN,"['Horror', 'Thriller']",['United States'],"['Expedition Films', 'Kismet Entertainment Gro...",4.6,NaN,NaN,NaN,"$1,300,000",NaN,NaN,NaN
2,tt0114978,Derevyannaya komnata,NaN,2017.0,NaN,NaN,"['Vladimir Maslov', 'Evgeniy Yufit']","['Vladimir Maslov', 'Evgeniy Yufit']","['Tatyana Verkhovskaya', 'Vladimir Maslov', 'I...",['Drama'],['Russia'],['Mzhalala Film'],6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0137204,Joe Finds Grace,NaN,2017.0,NaN,NaN,['Anthony Harrison'],['Anthony Harrison'],"['Anthony Harrison', 'C. Ernst Harth', 'Ellie ...","['Animation', 'Adventure', 'Comedy']",['Canada'],['Bradeway Pictures'],8.4,NaN,NaN,Awards\n2 wins,NaN,NaN,NaN,NaN
4,tt0331314,Bunyan and Babe,NaN,2017.0,PG,1h 24min,['Louis Ross'],"['Michael A. Nickles', 'Julia Wall']","['John Goodman', 'Jeff Foxworthy', 'Kelsey Gra...","['Animation', 'Adventure', 'Comedy']","['United States', 'India']","['Exodus Film Group', 'Toonz Media Group']",5.0,NaN,NaN,NaN,NaN,NaN,NaN,"$72,060"


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17969 entries, 0 to 17968
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   imdb_id            17968 non-null  object 
 1   title              17934 non-null  object 
 2   original_title     745 non-null    object 
 3   year               17930 non-null  float64
 4   certificate        2800 non-null   object 
 5   duration           2771 non-null   object 
 6   directors          17349 non-null  object 
 7   writers            15537 non-null  object 
 8   stars              11643 non-null  object 
 9   genres             17113 non-null  object 
 10  countries          17595 non-null  object 
 11  companies          12756 non-null  object 
 12  imdb_rating        9812 non-null   float64
 13  metascore          834 non-null    float64
 14  popularity         203 non-null    object 
 15  awards             4475 non-null   object 
 16  budget             499

---
## Número de películas

En primer lugar verificar si hay una discrepancia entre el número de películas obtenidas de title_basics.tsv de IMDb
y el número de películas resultantes del scrapping.

In [8]:
# Cargar las películas de la base de datos de IMDb en una lista

imdb_ids = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]
imdb_ids = list(imdb_ids['tconst'])
len(imdb_ids)

17967

In [9]:
scrap_ids = list(movies['imdb_id'])
len(scrap_ids)

17969

---
## 1. Bd de IMDb: `title_basics`

### Variable `runtimeMinutes`  

In [10]:
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8084314 entries, 0 to 8084313
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         float64
 5   startYear       float64
 6   endYear         float64
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(3), object(6)
memory usage: 555.1+ MB


#### Convertir `runtimeMinutes` en float
La siguiente función convierte los valores en float si se puede de manera natural, y si no se puede los imprime y convierte a nulo.

In [11]:
def to_float(n):
    try:
        n = float(n)
        return n
    except:
        print(n)
        return np.nan

In [12]:
title_basics.runtimeMinutes = title_basics.runtimeMinutes.apply(to_float)
title_basics.runtimeMinutes

Reality-TV
Documentary
Talk-Show
Game-Show
Reality-TV
Animation,Comedy,Family
Reality-TV


0           1.0
1           5.0
2           4.0
3          12.0
4           1.0
           ... 
8084309     NaN
8084310     NaN
8084311     NaN
8084312    27.0
8084313    10.0
Name: runtimeMinutes, Length: 8084314, dtype: float64

#### Eliminar de `title_basics` los registros con outliers en `runtimeMinutes`

In [13]:
# Buscar outliers mayores de 1.000 minutos y menores 15
# El resltado es un listado de películas fake, cortos o compendios de pelis.
# Eliminar todos estos registros del scrapping

title_basics = title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)]

title_basics = title_basics[~((title_basics.runtimeMinutes<15) | (title_basics.runtimeMinutes>500))]
title_basics.shape

(17953, 9)

In [14]:
# Una vez eliminados los outliers estos son los estadísticos:

title_basics.runtimeMinutes.describe().round(2)

count    14098.00
mean        89.84
std         26.29
min         30.00
25%         74.00
50%         89.00
75%        102.00
max        495.00
Name: runtimeMinutes, dtype: float64

---
## 2. Bd de IMDb: `title_ratings`

In [15]:
title_ratings = pd.read_csv('../data/imdb/title_ratings.tsv', sep='\t')
print(title_ratings.shape)
title_ratings.head()

(1171920, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,1722
1,tt0000002,6.0,211
2,tt0000003,6.5,1484
3,tt0000004,6.1,124
4,tt0000005,6.2,2286



---
## 3. Bd propia escrapeada del portal IMDb: `movies`

## `year`
La base de datos de IMDb descargada tiene algunas discrepancias en cuanto al año de las películas, no coindice el año en la base de datos descargada de IMDb con el año del portal web, en el caso del año 2019 ocurre en 12 registros. Existe algún caso en el que tampoco coincide el año de la película en el portal Metacritic. Hay varias formas de afrontar estas discrepancias, en este caso, y por simplificación, utilizaré el año de la base de datos de IMDb.
Hay también algunos registros en la base de datos con valores nulos para el año y demás columnas, y serán eliminados (47 registors en total).

In [16]:
# Películas escrapeadas con año diferente
len(movies[(movies['year']!=anno) & movies['year'].notnull()])

11

In [17]:
movies[(movies['year']!=anno) & movies['year'].notnull()]

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,countries,companies,imdb_rating,metascore,popularity,awards,budget,gross_us_canada,opening_us_canada,gross_world
2640,tt3385704,Sugar!,NaN,2016.0,NaN,NaN,['Shari Berman'],['Leora Kalish'],"['Alice Ripley', 'Robert Clohessy', 'Kathryn D...","['Drama', 'Music']",['United States'],"['Silver Phoenix Entertainment', 'Sugar Film']",4.3,NaN,NaN,Awards\n9 wins,NaN,NaN,NaN,NaN
2772,tt3576242,Obscura,NaN,2020.0,NaN,NaN,['Robert Thomas Preston'],"['Andrew John', 'Micah McCaw', 'Robert Thomas ...","['Andrew John', 'Robert Thomas Preston', 'Mark...","['Crime', 'Drama', 'Thriller']",['United States'],"['Wooden Finger Studios', 'Cora B. Productions...",4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3089,tt3946960,Chapter & Verse,NaN,2016.0,R,1h 37min,['Jamal Joseph'],"['Jamal Joseph', 'Daniel Beaty']","['Daniel Beaty', 'Loretta Devine', 'Omari Hard...",['Drama'],['United States'],"['The Bubble Factory', 'Harlem Film Company']",6.4,70.0,NaN,NaN,NaN,"$84,147","$31,331","$84,147"
6274,tt5934552,Powrót do Legolandu,NaN,2021.0,NaN,NaN,['Konrad Aksinowicz'],['Konrad Aksinowicz'],"['Anna Majcher', 'Maciej Stuhr', 'Katarzyna Wa...",['Drama'],['Poland'],"['Chroma Pro', 'Miasto Aniolów', 'Coloroffon']",NaN,NaN,NaN,Awards\n1 nomination,NaN,NaN,NaN,NaN
11288,tt6940686,Neron,NaN,2016.0,PG-13,1h 40min,"['Mitesh Kumar Patel', 'Sam Son']","['Julian Jay Burton', 'Lekhraj Patel']","['Kacey Clarke', 'Eric Roberts', 'Caite Upton']","['Action', 'Drama', 'Horror']",['United States'],"['Ad Men at Work', 'Applied Art Productions', ...",2.6,NaN,NaN,NaN,"$5,500,000",NaN,NaN,NaN
11709,tt7015334,The Limit Of,NaN,2018.0,NaN,NaN,['Alan Mulligan'],['Alan Mulligan'],"[""Laurence O'Fuarain"", 'Joanne Brennan', 'Des ...","['Action', 'Drama']",['Ireland'],['Ignition Film Productions'],6.1,NaN,NaN,Awards\n1 win & 3 nominations,NaN,NaN,NaN,"$5,909"
11862,tt7048500,Legends of Carpathians,NaN,2018.0,NaN,NaN,['Serg Skobun'],"['Valeriy Kharchyshyn', 'Sergiy Akhrimenko', '...",NaN,"['Action', 'Adventure', 'History']",['Ukraine'],['Magnat Media'],5.2,NaN,NaN,NaN,"$5,000,000",NaN,NaN,"$59,421"
11998,tt7071268,Desobediencia: o cómo Entrenar Gallos de Pelea,NaN,2019.0,NaN,NaN,['Tuchí Ortiz'],['Tuchí Ortiz'],"['Brayan Mulato', 'Eduardo Ortiz', 'Sergio Eli...",['Thriller'],['Colombia'],NaN,NaN,NaN,NaN,NaN,"$30,000",NaN,NaN,NaN
12392,tt7138200,Desobediencia: o cómo Entrenar Gallos de Pelea,NaN,2019.0,NaN,NaN,['Tuchí Ortiz'],['Tuchí Ortiz'],"['Brayan Mulato', 'Eduardo Ortiz', 'Sergio Eli...",['Thriller'],['Colombia'],NaN,NaN,NaN,NaN,NaN,"$30,000",NaN,NaN,NaN
13827,tt7390768,Comic,NaN,2018.0,NaN,NaN,['Sam Callis'],['Terry Alderton'],NaN,['Documentary'],['United Kingdom'],['Breakneck Films'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Eliminar del df escrapeado las películas de años posteriores a 2019
(son pelis futuras que se han colado)

In [18]:
movies = movies[~(movies['year']>2019)]
len(movies)

17967

#### Eliminar del df escrapeado las películas sin año
(son pelis sin información o url en el portal)

In [19]:
# Películas escrapeadas con año null, no tienen casi información, serán eliminadas
movies = movies[movies['year'].notnull()]
len(movies)

17928

## `imdb_rating`
Esta variable será sustituida por la de la base de datos `title_rating` descargada de IMDb

In [20]:
movies[['imdb_rating']].describe().round(2)

,imdb_rating
count,9811.00
mean,6.24
std,1.47
min,1.00
25%,5.30
50%,6.40
75%,7.30
max,10.00


## `metascore`
Comparing to: https://www.metacritic.com/feature/best-movies-released-in-2019

Hay pequeñas discrepancias en el listado de películas, debido a las fechas de estreno. Metacritic considera la fecha de estreno en USA. IMDb la fecha de estreno mundial. Pero sí que coinciden las puntuaciones y la mayoría de películas asignadas a 2019.

In [21]:
movies[['imdb_id', 'title', 'original_title', 'imdb_rating', 'metascore', 'year']][movies.metascore>=82].sort_values('metascore', ascending=False)

,imdb_id,title,original_title,imdb_rating,metascore,year
4086,tt4925292,Lady Bird,NaN,7.4,94.0,2017.0
4203,tt5013056,Dunkerque,Dunkirk,7.8,94.0,2017.0
5251,tt5598102,Caras y lugares,Visages villages,7.9,94.0,2017.0
5628,tt5726616,Call Me by Your Name,NaN,7.9,93.0,2017.0
7460,tt6217608,The Rider,NaN,7.5,92.0,2017.0
...,...,...,...,...,...,...
4163,tt4978710,Marjorie Prime,NaN,6.3,82.0,2017.0
13087,tt7259986,Ava,NaN,6.6,82.0,2017.0
13453,tt7320560,Cuba a través de la cámara,Cuba and the Cameraman,8.3,82.0,2017.0
2685,tt3450958,La guerra del planeta de los simios,War for the Planet of the Apes,7.4,82.0,2017.0


In [22]:
np.sort(movies.metascore.dropna().unique())

array([12., 13., 14., 15., 16., 17., 18., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94.])

In [23]:
movies.metascore.describe()

count    834.000000
mean      60.377698
std       16.537782
min       12.000000
25%       49.000000
50%       63.000000
75%       73.000000
max       94.000000
Name: metascore, dtype: float64

## `popularity`
Valor que vamos a descartar porque es muy volátil, cambia continuamente. Así, no guarda relación temporal con la taquilla que es el objeto de estudio.

In [24]:
# Convertirlo a número
movies['popularity'] = movies.popularity.str.replace(',', '').astype(float)

In [25]:
movies.popularity.describe()

count     203.000000
mean     2435.655172
std      1492.145541
min        68.000000
25%      1096.500000
50%      2312.000000
75%      3838.500000
max      4999.000000
Name: popularity, dtype: float64

## `directors`, `writers`, `stars`, `countries`, `companies`
Convertir a: comma separated values

In [26]:
movies['directors'] = movies['directors'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [27]:
movies['writers'] = movies['writers'].str.replace('[', '', regex=False) \
                                     .str.replace(']', '', regex=False) \
                                     .str.replace("'", "", regex=False) \
                                     .str.replace(", ", ",", regex=False)

In [28]:
movies['stars'] = movies['writers'].str.replace('[', '', regex=False) \
                                   .str.replace(']', '', regex=False) \
                                   .str.replace("'", "", regex=False) \
                                   .str.replace(", ", ",", regex=False)

In [29]:
movies['countries'] = movies['countries'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

In [30]:
movies['companies'] = movies['companies'].str.replace('[', '', regex=False) \
                                         .str.replace(']', '', regex=False) \
                                         .str.replace("'", "", regex=False) \
                                         .str.replace(", ", ",", regex=False)

## `opening_us_canada`, `gross_us_canada`, `gross_world`
Convertir estos valores a números, eliminando el signo del dólar

In [31]:
def remove_dolar(amount):
    if str(amount) == 'nan':
        return np.nan
    else:        
        return float(amount.replace(',', '')[1:]) 

In [32]:
movies['gross_us_canada'] = movies['gross_us_canada'].apply(remove_dolar)
movies['gross_us_canada']

0        3624.0
1           NaN
2           NaN
3           NaN
4           NaN
          ...  
17964       NaN
17965       NaN
17966       NaN
17967       NaN
17968       NaN
Name: gross_us_canada, Length: 17928, dtype: float64

In [33]:
movies['opening_us_canada'] = movies['opening_us_canada'].apply(remove_dolar)
movies['opening_us_canada']

0        512.0
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
17964      NaN
17965      NaN
17966      NaN
17967      NaN
17968      NaN
Name: opening_us_canada, Length: 17928, dtype: float64

In [34]:
movies['gross_world'] = movies['gross_world'].apply(remove_dolar)
movies['gross_world']

0         3624.0
1            NaN
2            NaN
3            NaN
4        72060.0
          ...   
17964        NaN
17965        NaN
17966        NaN
17967        NaN
17968        NaN
Name: gross_world, Length: 17928, dtype: float64

## `budget`
### Normalización `budget` a dólares
Son necesarios varios procesos para tener un valor válido de la variable `budget`:
- Separar, con regex, el texto que corresponde a la moneda y el que corresponde a la cantidad (ej. '€195,000'). Estos dos textos serán dos nuevas columnas `budgetCurrency` y `budgetAmount`
- Crear un diccionario con la correspondencia del código usado para las monedas en IMDb y el código ISO
- Crear un archivo externo que contenga las tasas de cambio para cada moneda y año
- Crear una nueva columna `budgetInDolars` con el valor ya normalizado a dólares

In [35]:
import re

def clean_amount_currency(raw_amount):
    
    if str(raw_amount) == 'nan' :
        return (np.nan, np.nan)
    
    # Diccionario con correspondencia del símbolo de moneda de IMDb y el ISO de monedas
    # Monedas de 2019 sin tasas de cambio: 'LVL'
    currency_to_iso = {'$':'USA', 'A$':'AUS', 'ZAR':'ZAF', '₹':'IND', '€':'EU27_2020',
                       'RUR':'RUS', 'R$':'BRA', 'CN¥':'CHN', 'TRL':'TUR', 'CA$':'CAN',
                       'NPR':'NPR', '£':'GBR', '¥':'JPN', 'HUF':'HUN', 'HRK':'HRV',
                       'NOK':'NOR', 'MNT':'MNT', 'CHF':'CHE', 'MX$':'MEX', 'UAH':'UAH',
                       'HK$':'HKG', 'PHP':'PHP', 'PLN':'POL', 'EGP':'EGP' , 'IRR':'IRR',
                       'NZ$':'NZL', 'AED':'AED', 'DKK':'DNK', 'NGN':'NGN', 'SEK':'SWE',
                       'BDT':'BDT', 'MYR':'MYR', 'CZK':'CZE', 'ARS':'ARG', 'PKR':'PKR',
                       'GEL':'GEL', 'NT$':'TWD', '₪':'ISR', '₫':'VND', '₩':'KOR', 'THB':'THB',
                       'RON':'ROU', 'PTE':'PRT', 'TTD':'TTD', 'MVR':'MVR', 'JOD':'JOD',
                       'ISK':'ISL', 'COP':'COL', 'LKR':'LKR', 'BSD':'BSD', 'BGL':'BGR',
                       'KZT':'KZT', 'KES':'KES', 'TZS':'TZS', 'DOP':'DOP', 'TJS':'TJS',
                       'TND':'TND', 'MUR':'MUR', 'SGD':'SGD', 'MOP':'MOP', 'IDR':'IDN',
                       'ETB':'ETB', 'CUP':'CUP', 'BYR':'BYR', 'MMK':'MMK', 'NAD':'NAD',
                       'PEN':'PER', 'MKD':'MKD', 'UGX':'UGX', 'BAM':'BAM', 'ALL':'ALL',
                       'KGS':'KGS', 'GHC':'GHS'
                      }
    
    raw_amount = raw_amount.replace(',', '')
    
    for item in re.finditer(r"^(\D+)(\d+)$", raw_amount):
        try:
            currency = currency_to_iso[item.group(1).strip()]
        except:
            currency = str(np.nan)
        amount = float(item.group(2))

    return(currency, amount)



# Dividir la info de budget en dos nuevas columnas (cantidad y moneda)
movies['budgetCurrency'], movies['budgetAmount'] = zip(*movies['budget'].map(clean_amount_currency))
movies[['title', 'budget', 'budgetAmount', 'budgetCurrency']].dropna()

# importar archivo csv de tasas de cambio
exchange_rates=pd.read_csv('../data/OECD/DP_LIVE_16072021155836489.csv')

# Seleccionar las tasas de cambio para el año del dataset
exchange_rates_anno = exchange_rates[exchange_rates.TIME == anno]

# Merge tasas de cambio con df movies, usando como primary key el código ISO de la moneda
movies = pd.merge(movies, exchange_rates_anno[['LOCATION','Value']], how='left', left_on='budgetCurrency', right_on='LOCATION')
movies['budgetInDollars'] = movies['budgetAmount'] / movies['Value']

movies.head()

,imdb_id,title,original_title,year,certificate,duration,directors,writers,stars,genres,...,awards,budget,gross_us_canada,opening_us_canada,gross_world,budgetCurrency,budgetAmount,LOCATION,Value,budgetInDollars
0,tt0100275,La Telenovela Errante,NaN,2017.0,NaN,NaN,"Raoul Ruiz,Valeria Sarmiento","Pía Rey,Raoul Ruiz","Pía Rey,Raoul Ruiz","['Comedy', 'Drama', 'Fantasy']",...,Awards\n1 win & 4 nominations,NaN,3624.0,512.0,3624.0,NaN,NaN,NaN,NaN,NaN
1,tt0112502,Bigfoot,NaN,2017.0,NaN,NaN,Mc Jones,NaN,NaN,"['Horror', 'Thriller']",...,NaN,"$1,300,000",NaN,NaN,NaN,USA,1300000.0,USA,1.0,1300000.0
2,tt0114978,Derevyannaya komnata,NaN,2017.0,NaN,NaN,"Vladimir Maslov,Evgeniy Yufit","Vladimir Maslov,Evgeniy Yufit","Vladimir Maslov,Evgeniy Yufit",['Drama'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0137204,Joe Finds Grace,NaN,2017.0,NaN,NaN,Anthony Harrison,Anthony Harrison,Anthony Harrison,"['Animation', 'Adventure', 'Comedy']",...,Awards\n2 wins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0331314,Bunyan and Babe,NaN,2017.0,PG,1h 24min,Louis Ross,"Michael A. Nickles,Julia Wall","Michael A. Nickles,Julia Wall","['Animation', 'Adventure', 'Comedy']",...,NaN,NaN,NaN,NaN,72060.0,NaN,NaN,NaN,NaN,NaN


---
# Merge de tablas


In [36]:
imdb_merged = pd.merge(title_basics[(title_basics.titleType=='movie') & (title_basics.startYear==anno)] , title_ratings, on='tconst', how='left')
imdb_merged

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0100275,movie,The Wandering Soap Opera,La Telenovela Errante,0.0,2017.0,NaN,80.0,"Comedy,Drama,Fantasy",6.5,272.0
1,tt0112502,movie,Bigfoot,Bigfoot,0.0,2017.0,NaN,NaN,"Horror,Thriller",4.5,33.0
2,tt0114978,movie,Derevyannaya komnata,Derevyannaya komnata,0.0,2017.0,NaN,65.0,Drama,6.6,55.0
3,tt0137204,movie,Joe Finds Grace,Joe Finds Grace,0.0,2017.0,NaN,83.0,"Adventure,Animation,Comedy",8.4,277.0
4,tt0331314,movie,Bunyan and Babe,Bunyan and Babe,0.0,2017.0,NaN,84.0,"Adventure,Animation,Comedy",4.9,350.0
...,...,...,...,...,...,...,...,...,...,...,...
17948,tt9910238,movie,Be 5min. vede,Be 5min. vede,0.0,2017.0,NaN,80.0,"Comedy,Romance",3.8,13.0
17949,tt9913594,movie,Bacchanalia,Bacchanalia,0.0,2017.0,NaN,72.0,"Drama,Mystery,Thriller",NaN,NaN
17950,tt9914642,movie,Albatross,Albatross,0.0,2017.0,NaN,97.0,Documentary,8.1,32.0
17951,tt9916186,movie,Illenau - die Geschichte einer ehemaligen Heil...,Illenau - die Geschichte einer ehemaligen Heil...,0.0,2017.0,NaN,84.0,Documentary,NaN,NaN


### Renombrar y seleccionar columnas de ambos datasets

In [37]:
imdb_merged.rename(columns={'primaryTitle': 'englishTitle',
                            'tconst': 'imdbId',
                            'averageRating': 'ratingImdb'
                          }, inplace=True)

In [38]:
imdb_merged.columns

Index(['imdbId', 'titleType', 'englishTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres', 'ratingImdb',
       'numVotes'],
      dtype='object')

In [39]:
movies.rename(columns={'imdb_id': 'imdbId',
                       'title': 'spanishTitle',
                       'budget': 'budgetString',
                       'budgetInDollars': 'budget',
                       'gross_us_canada': 'grossUsCanada',
                       'opening_us_canada': 'openingUsCanada',
                       'gross_world': 'grossWorld',
                      }, inplace=True)

In [40]:
movies.columns

Index(['imdbId', 'spanishTitle', 'original_title', 'year', 'certificate',
       'duration', 'directors', 'writers', 'stars', 'genres', 'countries',
       'companies', 'imdb_rating', 'metascore', 'popularity', 'awards',
       'budgetString', 'grossUsCanada', 'openingUsCanada', 'grossWorld',
       'budgetCurrency', 'budgetAmount', 'LOCATION', 'Value', 'budget'],
      dtype='object')

In [42]:
# Selección de columnas de imdb
imdb_merged = imdb_merged[['imdbId', 'englishTitle', 'originalTitle', 'isAdult', 'runtimeMinutes', 'genres', 'ratingImdb', 'numVotes']]

In [44]:
# Selección de columnas del escrapeado
movies = movies[['imdbId', 'spanishTitle', 'year', 'certificate',
                'directors', 'writers', 'stars', 'countries', 'companies',
                'metascore', 'awards', 'budget', 'grossUsCanada',
                'openingUsCanada', 'grossWorld']]

### Merge final para este año

In [45]:
movies_merged = pd.merge(imdb_merged, movies, on='imdbId', how='inner')
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17914 entries, 0 to 17913
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           17914 non-null  object 
 1   englishTitle     17914 non-null  object 
 2   originalTitle    17914 non-null  object 
 3   isAdult          17914 non-null  float64
 4   runtimeMinutes   14096 non-null  float64
 5   genres           17069 non-null  object 
 6   ratingImdb       9772 non-null   float64
 7   numVotes         9772 non-null   float64
 8   spanishTitle     17914 non-null  object 
 9   year             17914 non-null  float64
 10  certificate      2799 non-null   object 
 11  directors        17333 non-null  object 
 12  writers          15521 non-null  object 
 13  stars            15521 non-null  object 
 14  countries        17579 non-null  object 
 15  companies        12746 non-null  object 
 16  metascore        834 non-null    float64
 17  awards      

In [47]:
# Reordenar columnas

movies_merged = movies_merged[['imdbId', 'year', 'spanishTitle', 'originalTitle',
                               'englishTitle', 'ratingImdb', 'numVotes', 'metascore',
                               'isAdult', 'certificate', 'runtimeMinutes', 'genres', 'directors',
                               'writers', 'stars', 'countries', 'companies', 'awards',
                               'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld']]

In [48]:
movies_merged

,imdbId,year,spanishTitle,originalTitle,englishTitle,ratingImdb,numVotes,metascore,isAdult,certificate,...,directors,writers,stars,countries,companies,awards,budget,grossUsCanada,openingUsCanada,grossWorld
0,tt0100275,2017.0,La Telenovela Errante,La Telenovela Errante,The Wandering Soap Opera,6.5,272.0,NaN,0.0,NaN,...,"Raoul Ruiz,Valeria Sarmiento","Pía Rey,Raoul Ruiz","Pía Rey,Raoul Ruiz",Chile,"Poetastros,RR Producciones",Awards\n1 win & 4 nominations,NaN,3624.0,512.0,3624.0
1,tt0112502,2017.0,Bigfoot,Bigfoot,Bigfoot,4.5,33.0,NaN,0.0,NaN,...,Mc Jones,NaN,NaN,United States,"Expedition Films,Kismet Entertainment Group",NaN,1300000.0,NaN,NaN,NaN
2,tt0114978,2017.0,Derevyannaya komnata,Derevyannaya komnata,Derevyannaya komnata,6.6,55.0,NaN,0.0,NaN,...,"Vladimir Maslov,Evgeniy Yufit","Vladimir Maslov,Evgeniy Yufit","Vladimir Maslov,Evgeniy Yufit",Russia,Mzhalala Film,NaN,NaN,NaN,NaN,NaN
3,tt0137204,2017.0,Joe Finds Grace,Joe Finds Grace,Joe Finds Grace,8.4,277.0,NaN,0.0,NaN,...,Anthony Harrison,Anthony Harrison,Anthony Harrison,Canada,Bradeway Pictures,Awards\n2 wins,NaN,NaN,NaN,NaN
4,tt0331314,2017.0,Bunyan and Babe,Bunyan and Babe,Bunyan and Babe,4.9,350.0,NaN,0.0,PG,...,Louis Ross,"Michael A. Nickles,Julia Wall","Michael A. Nickles,Julia Wall","United States,India","Exodus Film Group,Toonz Media Group",NaN,NaN,NaN,NaN,72060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17909,tt9910238,2017.0,Be 5min. vede,Be 5min. vede,Be 5min. vede,3.8,13.0,NaN,0.0,NaN,...,Dalius Pletniovas,Dalius Pletniovas,Dalius Pletniovas,Lithuania,NaN,NaN,NaN,NaN,NaN,NaN
17910,tt9913594,2017.0,Bacchanalia,Bacchanalia,Bacchanalia,NaN,NaN,NaN,0.0,NaN,...,Emma Jouannet,Emma Jouannet,Emma Jouannet,France,NaN,NaN,NaN,NaN,NaN,NaN
17911,tt9914642,2017.0,Albatross,Albatross,Albatross,8.1,32.0,NaN,0.0,NaN,...,Chris Jordan,"Chris Jordan,Victoria Sloan Jordan","Chris Jordan,Victoria Sloan Jordan",United States,NaN,NaN,NaN,NaN,NaN,NaN
17912,tt9916186,2017.0,Illenau - die Geschichte einer ehemaligen Heil...,Illenau - die Geschichte einer ehemaligen Heil...,Illenau - die Geschichte einer ehemaligen Heil...,NaN,NaN,NaN,0.0,NaN,...,Frank König,"David Nathan,Helmut Schiffner,Andree Steinke","David Nathan,Helmut Schiffner,Andree Steinke",Germany,Visiris Videoproduktion,NaN,NaN,NaN,NaN,NaN


## Feature engineering. Columnas `profit` y `roi`
Los estadísticos están distorsionados por los valores nulos que se eliminarán después

In [51]:
movies_merged['profit'] = movies_merged.grossWorld - movies_merged.budget
movies_merged['profit'].describe()

<ipython-input-51-7363a4f7ae40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_merged['profit'] = movies_merged['grossWorld'] - movies_merged['budget']


count    7.120000e+02
mean     2.911790e+07
std      1.236476e+08
min     -1.945609e+08
25%     -2.810427e+06
50%     -3.897311e+05
75%      1.353116e+06
max      1.104435e+09
Name: profit, dtype: float64

In [52]:
movies_merged['roi'] = (movies_merged.grossWorld - movies_merged.budget) / movies_merged.budget
movies_merged['roi'].describe()

<ipython-input-52-84cf0a39417a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_merged['roi'] = (movies_merged.grossWorld - movies_merged.budget) / movies_merged.budget


count       712.000000
mean       1205.064283
std       24369.967810
min          -0.999962
25%          -0.957183
50%          -0.655481
75%           0.658416
max      625118.849262
Name: roi, dtype: float64

---
# Inspección inicial de correlaciones
## Registros no nulos con los que hacer las correlaciones de valoraciones y recaudación
Valoraciones: imdb_rating, metascore

Recaudación: budget, gross_world


In [55]:
print('Registros con ratingImdb:', len(movies_merged[movies_merged.ratingImdb.notnull()]))
print('Registros con metascore:', len(movies_merged[movies_merged.metascore.notnull()]))

print('Registros con budget:', len(movies_merged[movies_merged.budget.notnull()]))
print('Registros con grossUsCanada:', len(movies_merged[movies_merged.grossUsCanada.notnull()]))
print('Registros con openingUsCanada:', len(movies_merged[movies_merged.openingUsCanada.notnull()]))
print('Registros con grossWorld:', len(movies_merged[movies_merged.grossWorld.notnull()]))

print('\nRegistros con las rating, metascore, budget, grossWorld:',len(movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]))

Registros con ratingImdb: 9772
Registros con metascore: 834
Registros con budget: 4961
Registros con grossUsCanada: 689
Registros con openingUsCanada: 612
Registros con grossWorld: 2923

Registros con las rating, metascore, budget, grossWorld: 238


In [56]:
movies_merged = movies_merged[movies_merged.budget.notnull() & movies_merged.grossWorld.notnull() & movies_merged.ratingImdb.notnull() & movies_merged.metascore.notnull()]
movies_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238 entries, 7 to 16064
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdbId           238 non-null    object 
 1   year             238 non-null    float64
 2   spanishTitle     238 non-null    object 
 3   originalTitle    238 non-null    object 
 4   englishTitle     238 non-null    object 
 5   ratingImdb       238 non-null    float64
 6   numVotes         238 non-null    float64
 7   metascore        238 non-null    float64
 8   isAdult          238 non-null    float64
 9   certificate      228 non-null    object 
 10  runtimeMinutes   238 non-null    float64
 11  genres           238 non-null    object 
 12  directors        238 non-null    object 
 13  writers          238 non-null    object 
 14  stars            238 non-null    object 
 15  countries        238 non-null    object 
 16  companies        238 non-null    object 
 17  awards        

## Correlaciones de valoraciones y recaudación

In [58]:
movies_merged.describe().round(2)

,year,ratingImdb,numVotes,metascore,isAdult,runtimeMinutes,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
count,238.0,238.00,238.00,238.00,238.0,238.00,2.380000e+02,2.040000e+02,1.990000e+02,2.380000e+02,2.380000e+02,238.00
mean,2017.0,6.40,85711.59,57.48,0.0,108.91,3.636120e+07,5.215012e+07,1.608255e+07,1.252772e+08,8.891599e+07,720.24
std,0.0,0.92,136162.55,17.94,0.0,15.72,5.351165e+07,9.214008e+07,3.107564e+07,2.404122e+08,1.994857e+08,11012.89
min,2017.0,2.90,39.00,12.00,0.0,74.00,2.260000e+00,1.377000e+03,6.240000e+02,2.697000e+03,-7.217141e+07,-1.00
25%,2017.0,5.82,5952.00,43.25,0.0,97.00,5.462101e+06,8.894022e+05,6.249700e+04,2.604310e+06,-3.553721e+06,-0.59
50%,2017.0,6.50,26891.00,58.00,0.0,107.00,1.800000e+07,1.821984e+07,3.372565e+06,2.341184e+07,5.515741e+06,0.51
75%,2017.0,7.00,103051.00,72.75,0.0,118.75,3.800000e+07,5.480627e+07,1.825462e+07,1.257620e+08,7.571949e+07,2.90
max,2017.0,8.60,673008.00,94.00,0.0,164.00,3.170000e+08,6.201814e+08,2.200096e+08,1.332699e+09,1.104435e+09,169901.90


In [59]:
method = 'pearson'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2017 Corr Método: pearson


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.739971,0.095390,0.194421,0.208720,0.022149
metascore,0.739971,1.000000,-0.052913,0.064587,0.092031,0.042519
budget,0.095390,-0.052913,1.000000,0.811006,0.709144,-0.044475
grossWorld,0.194421,0.064587,0.811006,1.000000,0.987609,-0.033801
profit,0.208720,0.092031,0.709144,0.987609,1.000000,-0.028805
roi,0.022149,0.042519,-0.044475,-0.033801,-0.028805,1.000000


In [60]:
method = 'kendall'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2017 Corr Método: kendall


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.562327,-0.024464,0.112113,0.150048,0.199235
metascore,0.562327,1.000000,-0.156972,-0.016715,0.069003,0.107219
budget,-0.024464,-0.156972,1.000000,0.589666,0.284837,0.217913
grossWorld,0.112113,-0.016715,0.589666,1.000000,0.696699,0.630110
profit,0.150048,0.069003,0.284837,0.696699,1.000000,0.664220
roi,0.199235,0.107219,0.217913,0.630110,0.664220,1.000000


In [61]:
method = 'spearman'

print(anno, 'Corr Método:', method)

movies_merged[['ratingImdb', 'metascore', 'budget', 'grossWorld', 'profit', 'roi']].corr(method = method)

2017 Corr Método: spearman


,ratingImdb,metascore,budget,grossWorld,profit,roi
ratingImdb,1.000000,0.740157,-0.034254,0.166951,0.220620,0.291851
metascore,0.740157,1.000000,-0.225981,-0.020739,0.092177,0.157526
budget,-0.034254,-0.225981,1.000000,0.771269,0.440877,0.311376
grossWorld,0.166951,-0.020739,0.771269,1.000000,0.846103,0.810201
profit,0.220620,0.092177,0.440877,0.846103,1.000000,0.866241
roi,0.291851,0.157526,0.311376,0.810201,0.866241,1.000000


In [62]:
movies_merged[['ratingImdb', 'metascore', 'budget', 'grossUsCanada', 'openingUsCanada', 'grossWorld', 'profit', 'roi']].corr()

,ratingImdb,metascore,budget,grossUsCanada,openingUsCanada,grossWorld,profit,roi
ratingImdb,1.000000,0.739971,0.095390,0.211609,0.128390,0.194421,0.208720,0.022149
metascore,0.739971,1.000000,-0.052913,0.115186,0.054957,0.064587,0.092031,0.042519
budget,0.095390,-0.052913,1.000000,0.740341,0.780573,0.811006,0.709144,-0.044475
grossUsCanada,0.211609,0.115186,0.740341,1.000000,0.942974,0.908517,0.896213,-0.015226
openingUsCanada,0.128390,0.054957,0.780573,0.942974,1.000000,0.879262,0.849813,-0.018876
grossWorld,0.194421,0.064587,0.811006,0.908517,0.879262,1.000000,0.987609,-0.033801
profit,0.208720,0.092031,0.709144,0.896213,0.849813,0.987609,1.000000,-0.028805
roi,0.022149,0.042519,-0.044475,-0.015226,-0.018876,-0.033801,-0.028805,1.000000


---
# Guardado dataset a pickle

In [63]:
import pickle

with open('../data/web_imdb_clean/movies_2017.pickle', 'wb') as f:
    pickle.dump(movies_merged, f)